In [ ]:
# Import dataset from Kaggle
! pip install -q kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d wcukierski/enron-email-dataset

In [ ]:
# Unzip dataset
! unzip enron-email-dataset.zip

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(r"/content/emails.csv")

In [ ]:
# Process data so that it only includes subject and header
emails = pd.DataFrame()
emails["content"] = df["message"].str.extract("Subject: (.+)\n")[0].str.strip().fillna("") +\
"\n" +  df["message"].str.split("\n\n", 1).str[1].str.strip().fillna("")

In [ ]:
import string
import nltk
import math
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
# Example
x = emails.iloc[1000]["content"]
print(x)
print(set([token.lower() for token in nltk.word_tokenize(x)]))


Lucy,

Here is the rentroll.

My only questions are about #18, #25, and #37 missed rent.  Any special 
reasons?

It looks like there are five vacancies #2,12,20a,35,40.  If you want to run 
an ad in the paper with a $50 discount that is fine.
I will write you a letter of recommendation.  When do you need it?  You can 
use me as a reference.  In the next two weeks we should really have a good 
idea whether the sale is going through.

Phillip
{'rentroll', 'fine', '18', 'me', 'ad', 'when', 'vacancies', 'only', 'as', 'five', 'use', 'any', 'here', 'lucy', 'reasons', 'to', 'will', 'next', 'whether', 'it', 'my', 'two', 'good', 'do', '.', 'of', 'need', 'there', 'is', 'going', 'an', 'the', '37', 'like', '2,12,20a,35,40', 'a', 'are', 'phillip', 'and', '50', 'should', 'discount', 'in', 'looks', '?', 'rent', 'run', 'write', 'i', 'reference', 'have', 'idea', 'want', 'can', 'paper', 'questions', 'through', '$', 'special', 'we', 'sale', 'that', '25', 'missed', 'recommendation', 'about', 'weeks', ','

In [ ]:
N_q = dict()
for i, content in enumerate(emails["content"]):
  if i % 10000 == 0:
    print(i, "completed") # Print current completion status
  for word in set([token.lower() for token in nltk.word_tokenize(content)]):
    word = word.lower()
    N_q[word] = N_q.setdefault(word, 0) + 1

In [ ]:
emails.info() # Get number of rows

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517401 entries, 0 to 517400
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   content  517401 non-null  object
dtypes: object(1)
memory usage: 3.9+ MB


In [ ]:
N = 517401
idf = dict()
for word in N_q:
  idf[word] = math.log((N - N_q[word] + 0.5) / (N_q[word] + 0.5)) + 1

In [ ]:
import json
with open('idf.json', 'w') as f:
    json.dump(idf, f)